# Comparison Using LFW Dataset - Facenet

Several feature extractor:
1. Face Embedding: Facenet
2. Face Embedding: VGG Face
3. Face Embedding: VGG Face - VGG16
4. Face Embedding: VGG Face - RESNET50
5. LBPH (Local Binary Pattern Histogram)

In [1]:
from scipy.spatial.distance import euclidean
from sklearn.metrics import accuracy_score, f1_score
from keras import backend as K
from feature_extractor.face_feature_extractor import FaceFeatureExtractor
import numpy as np
import cv2 as cv
from mtcnn.mtcnn import MTCNN
import matplotlib.pyplot as plt

/home/agusgun/anaconda3/envs/basic/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Read Dataset

In [2]:
import os

In [3]:
DIR_PATH = '../lfw/'
image_path_list = []
labels = []
name_dictionary = {}
counter = 0
for root, dirs, files in os.walk(DIR_PATH):
    for filename in files:
        person_name = ' '.join(filename.split('.')[0].split('_')[0:-1]) 
        file_path = os.path.join(root, filename)
        if person_name not in name_dictionary:
            counter += 1
            name_dictionary[person_name] = counter
        image_path_list.append(file_path)
        labels.append(name_dictionary[person_name])

In [4]:
print(len(labels))
print(len(image_path_list))

13233
13233


In [5]:
image_path_list[0:5]

['../lfw/Ryan_Newman/Ryan_Newman_0001.jpg',
 '../lfw/Dimitar_Berbatov/Dimitar_Berbatov_0001.jpg',
 '../lfw/Ed_Rendell/Ed_Rendell_0001.jpg',
 '../lfw/Joe_Crede/Joe_Crede_0001.jpg',
 '../lfw/Norman_Mailer/Norman_Mailer_0001.jpg']

## Shuffle It

In [6]:
import random

In [7]:
temp = list(zip(image_path_list, labels))
random.Random(0).shuffle(temp) # custom seed
image_path_list, labels = zip(*temp)

In [8]:
image_path_list[0:5]

('../lfw/Tommy_Maddox/Tommy_Maddox_0001.jpg',
 '../lfw/David_Millar/David_Millar_0001.jpg',
 '../lfw/Gregg_Popovich/Gregg_Popovich_0004.jpg',
 '../lfw/Shimon_Peres/Shimon_Peres_0001.jpg',
 '../lfw/Rudolph_Giuliani/Rudolph_Giuliani_0024.jpg')

In [9]:
labels[0:5]

(3038, 1517, 3115, 834, 1016)

In [10]:
name_dictionary['John Ashcroft']

482

# Experiment

In [11]:
FACENET_MODEL_PATH = 'model_data/facenet/20180402-114759'
VGGFACE_MODEL_PATH = 'model_data/vgg_face_weights.h5'

In [12]:
mtcnn_detector = MTCNN()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [13]:
def predict(image_representation_list, true_label, test_representation, min_distance):
    minimum_label = None
    minimum_distance = min_distance
    
    for idx, image_representation in enumerate(image_representation_list):
        distance = euclidean(image_representation, test_representation)
        if distance < minimum_distance:
            minimum_distance = distance
            minimum_label = true_label[idx]
    
    return minimum_label

In [14]:
def compute_score(predictions, labels):
    count_same = 0
    for idx, prediction in enumerate(predictions):
        if labels[idx] == prediction:
            count_same += 1
    return count_same / len(predictions)

## Facenet

Based on experiment Facenet have distance below 0.9 for euclidean if two image are the same person

In [15]:
feature_extractor = FaceFeatureExtractor(FACENET_MODEL_PATH, extractor_name='facenet')

Model directory: model_data/facenet/20180402-114759
Metagraph file: model-20180402-114759.meta
Checkpoint file: model-20180402-114759.ckpt-275
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Restoring parameters from model_data/facenet/20180402-114759/model-20180402-114759.ckpt-275


### Testing Starts Here

In [16]:
THRESHOLD = 0.9

In [17]:
image_representation_database = []
image_representation_labels = []
prediction_result = []

In [18]:
%%time
for idx, image_path in enumerate(image_path_list):
    if idx % 1000 == 0:
        print("Checkpoint", idx)
    img = cv.cvtColor(cv.imread(image_path), cv.COLOR_BGR2RGB)
    detection_result = mtcnn_detector.detect_faces(img)
    cropped_image = None
    for face in detection_result:
        face_bbox = face['box']
        x, y, w, h = face_bbox
        if x < 0:
            x = 0
        if y < 0:
            y = 0
        cropped_image = img[y:y+h, x:x+w]
        break
    
    if not cropped_image is None:
        feature_test = feature_extractor.extract_image(cropped_image)
        prediction = predict(image_representation_database, image_representation_labels, feature_test, THRESHOLD)
        if prediction == None:
            label = labels[idx]
            if label not in image_representation_labels:
                image_representation_labels.append(label)
                image_representation_database.append(feature_test)
                prediction_result.append(label)
            else: # false prediction
                prediction_result.append(-1) 
        else:
            prediction_result.append(prediction)
    else: # failed to detect faces
        prediction_result.append(-1)

Checkpoint 0
Checkpoint 1000
Checkpoint 2000
Checkpoint 3000
Checkpoint 4000
Checkpoint 5000
Checkpoint 6000
Checkpoint 7000
Checkpoint 8000
Checkpoint 9000
Checkpoint 10000
Checkpoint 11000
Checkpoint 12000
Checkpoint 13000
CPU times: user 2h 23min 46s, sys: 2h 9min 49s, total: 4h 33min 35s
Wall time: 56min 3s


In [19]:
image_path_list[37]

'../lfw/George_W_Bush/George_W_Bush_0233.jpg'

In [20]:
compute_score(prediction_result, labels)

0.5910224438902744

In [21]:
len(prediction_result)

13233

In [22]:
len(labels)

13233

In [23]:
# feature_extractor.extractor.close_session()

In [24]:
image_path_list = sorted(image_path_list)

In [25]:
image_path_list[31:50]

['../lfw/Abdullah_Gul/Abdullah_Gul_0001.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0002.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0003.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0004.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0005.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0006.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0007.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0008.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0009.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0010.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0011.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0012.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0013.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0014.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0015.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0016.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0017.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0018.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0019.jpg']

In [26]:
%%time
for idx, image_path in enumerate(image_path_list[31:32]):
    if idx % 1000 == 0:
        print("Checkpoint", idx)
    img = cv.cvtColor(cv.imread(image_path), cv.COLOR_BGR2RGB)
    detection_result = mtcnn_detector.detect_faces(img)
    cropped_image = None
    for face in detection_result:
        face_bbox = face['box']
        x, y, w, h = face_bbox
        if x < 0:
            x = 0
        if y < 0:
            y = 0
        cropped_image = img[y:y+h, x:x+w]
        break
    
    if not cropped_image is None:
        feature_test = feature_extractor.extract_image(cropped_image)
        prediction = predict(image_representation_database, image_representation_labels, feature_test, THRESHOLD)
        if prediction == None:
            label = labels[idx]
            if label not in image_representation_labels:
                image_representation_labels.append(label)
                image_representation_database.append(feature_test)
                prediction_result.append(label)
            else: # false prediction
                prediction_result.append(-1) 
        else:
            prediction_result.append(prediction)
    else: # failed to detect faces
        prediction_result.append(-1)

Checkpoint 0
CPU times: user 580 ms, sys: 428 ms, total: 1.01 s
Wall time: 181 ms


In [27]:
base_feature = feature_test.copy()
base_feature

array([-0.0405996 ,  0.02309781,  0.04990761,  0.000695  ,  0.0563635 ,
        0.0356246 , -0.04111306, -0.02143851,  0.09322309, -0.05160348,
        0.06762894, -0.04011588,  0.02762367, -0.02471023, -0.00265883,
       -0.04618575, -0.03693787,  0.03868102,  0.03121728, -0.0021389 ,
       -0.04605038,  0.02138399,  0.01972781, -0.10215669, -0.06483322,
        0.00368038,  0.00234469, -0.00917574, -0.03859585,  0.01891637,
        0.0174926 ,  0.08110538,  0.05090515,  0.02109467,  0.07272021,
       -0.05845511,  0.02789485, -0.06416457,  0.00062212,  0.02790122,
        0.06465936, -0.06553471, -0.01928616,  0.04141424,  0.00961436,
       -0.03530344, -0.05076744,  0.02497929, -0.05561506, -0.07793365,
       -0.0517925 , -0.04527229, -0.03888135,  0.06378752,  0.03055776,
       -0.00371595,  0.05760859, -0.05149259, -0.01650976, -0.00538056,
        0.02541253,  0.05328941, -0.01167208, -0.02779511,  0.00093871,
        0.03193406,  0.02281582, -0.0697334 , -0.03571099, -0.04

In [28]:
%%time
for idx, image_path in enumerate(image_path_list[31:50]):
    if idx % 1000 == 0:
        print("Checkpoint", idx)
    img = cv.cvtColor(cv.imread(image_path), cv.COLOR_BGR2RGB)
    detection_result = mtcnn_detector.detect_faces(img)
    cropped_image = None
    for face in detection_result:
        face_bbox = face['box']
        x, y, w, h = face_bbox
        if x < 0:
            x = 0
        if y < 0:
            y = 0
        cropped_image = img[y:y+h, x:x+w]
        break
    
    if not cropped_image is None:
        feature_test = feature_extractor.extract_image(cropped_image)
        distance = euclidean(feature_test, base_feature)
        print(distance)

Checkpoint 0
0.0
0.9304068088531494
0.6869785785675049
0.8082365989685059
0.7290970087051392
0.6848490238189697
0.43748414516448975
0.8663362264633179
0.9823824167251587
0.7026365399360657
0.7057827115058899
0.490217924118042
0.5257407426834106
0.653395414352417
0.6979514956474304
0.7552499175071716
0.609197199344635
0.7848309278488159
0.8689273595809937
CPU times: user 11.1 s, sys: 11.2 s, total: 22.3 s
Wall time: 3.4 s


In [29]:
%%time
scores = []
for idx, image_path in enumerate(image_path_list[0:30]):
    if idx % 1000 == 0:
        print("Checkpoint", idx)
    img = cv.cvtColor(cv.imread(image_path), cv.COLOR_BGR2RGB)
    detection_result = mtcnn_detector.detect_faces(img)
    cropped_image = None
    for face in detection_result:
        face_bbox = face['box']
        x, y, w, h = face_bbox
        if x < 0:
            x = 0
        if y < 0:
            y = 0
        cropped_image = img[y:y+h, x:x+w]
        break
    
    if not cropped_image is None:
        feature_test = feature_extractor.extract_image(cropped_image)
        distance = euclidean(feature_test, base_feature)
        print(distance)
        scores.append(distance)
print("AVG:", np.mean(scores))

Checkpoint 0
1.4091931581497192
1.5989396572113037
1.648654580116272
1.3955048322677612
1.460929274559021
1.4296987056732178
1.4172757863998413
1.4773813486099243
1.46343994140625
1.2700440883636475
1.5592588186264038
1.5224809646606445
1.114401936531067
1.562196969985962
1.490476369857788
1.2941749095916748
1.4539159536361694
1.4876363277435303
1.250456690788269
1.4930957555770874
1.5765728950500488
1.395493745803833
1.510777473449707
1.0394794940948486
1.4601694345474243
1.3177711963653564
1.2360012531280518
1.2233175039291382
1.2125657796859741
1.2607898712158203
AVG: 1.4010698239008585
CPU times: user 16.2 s, sys: 16.6 s, total: 32.8 s
Wall time: 4.96 s
